#### Setup

In [ ]:
from google.colab import drive
import sys

In [ ]:
drive.mount('/content/drive')
sys.path.append('/content/drive/Shareddrives/CMPE260/')

Mounted at /content/drive


In [ ]:
%run /content/drive/Shareddrives/CMPE260/rec_implementation.ipynb

cpu


#### Training

In [ ]:
train_env = Environment(train_matrix)

hits, dcgs = [], []
hits_all, dcgs_all = [], []
step, best_step = 0, 0
step, best_step, best_step_all = 0, 0, 0

users = np.random.permutation(apt_users)

In [ ]:
for u in tqdm.tqdm(users):
    print('user =', u)
    actions = []
    user, memory = train_env.reset(u)

    for t in range(int(train_matrix[u].sum())):
      try:
        # given current state, get action using actor network
        action_emb = policy_network(state_representation(user, memory))
        action = policy_network.get_action(user, torch.tensor(train_env.memory[user.detach().cpu().numpy().astype(int), :]), state_representation, action_emb,
            torch.tensor([item for item in train_env.available_books if item not in train_env.viewed_books]).long()
        )
      except Exception as e:
        print(e)

      # update replay buffer
      user, memory, reward, done = train_env.step(action, action_emb, replay_buffer)

      # update weights of action and critic network
      if len(replay_buffer) > params['batch_size']:
          update(params['batch_size'], params['gamma'], params['min_value'], params['max_value'], params['soft_tau'])

      # monitor training process by evaluating on only one user for efficiency
      if step % 100 == 0 and step > 0:
          hit, dcg = evaluation(policy_network, state_representation, train_env.memory, test_loader)

          hits.append(hit)
          dcgs.append(dcg)

          if np.mean(np.array([hit, dcg]) - np.array([hits[best_step], dcgs[best_step]])) >= 0:
              best_step = step // 100
              torch.save(policy_network.state_dict(), params['log_base_dir'] + 'policy_net.pth')
              torch.save(value_network.state_dict(), params['log_base_dir'] + 'value_net.pth')
              torch.save(state_representation.state_dict(), params['log_base_dir'] + 'state_repr.pth')

      # evaluate for best policy after every 1000 steps on the whole test set
      if step % 1000 == 0 and step > 0:
          hit, dcg = evaluation(policy_network, state_representation, train_env.memory, full_loader)

          hits_all.append(hit)
          dcgs_all.append(dcg)

          if np.mean(np.array([hit, dcg]) - np.array([hits_all[best_step_all], dcgs_all[best_step_all]])) >= 0:
              best_step_all = step // 1000
              torch.save(policy_network.state_dict(), params['log_base_dir'] + 'best_policy_net.pth')
              torch.save(value_network.state_dict(), params['log_base_dir'] + 'best_value_net.pth')
              torch.save(state_representation.state_dict(), params['log_base_dir'] + 'best_state_repr.pth')

      step += 1
    print(hits)
    print(dcgs)

  0%|          | 0/52 [00:00<?, ?it/s]

user = 144


  2%|▏         | 1/52 [00:06<05:44,  6.76s/it]

[0.984]
[0.49245854429314884]
user = 78


  4%|▍         | 2/52 [00:13<05:44,  6.89s/it]

...

[0.984, 0.992, 0.992, 0.984, 0.984, 0.992, 0.992, 0.984, 0.984, 0.984, 1.0, 0.992, 0.992, 0.992, 0.992, 0.992, 1.0, 0.984, 0.992, 0.992, 0.992, 0.992, 0.992, 1.0, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.984, 0.984, 0.984, 0.984, 0.984, 0.984, 0.984, 0.984, 0.984, 0.984, 0.984, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.984, 0.984]
[0.49245854429314884, 0.5019703776663528, 0.5082260916159203, 0.5130975501757558, 0.5013040682755924, 0.509983117336016, 0.5030687651372027, 0.4910859682113131, 0.49095953646710994, 0.4961424736980405, 0.4918502761825256, 0.506072271149805, 0.5063802974075646, 0.5085305863114334, 0.5045140704032558, 0.5009193860907593, 0.5143717027182738, 0.4947851760065075, 0.5022471043792723, 0.5021048438928243, 0.5070588873946523, 0.513012627219841, 0.5099358757855496, 0.5012

 98%|█████████▊| 51/52 [23:31<00:11, 11.40s/it]

[0.984, 0.992, 0.992, 0.984, 0.984, 0.992, 0.992, 0.984, 0.984, 0.984, 1.0, 0.992, 0.992, 0.992, 0.992, 0.992, 1.0, 0.984, 0.992, 0.992, 0.992, 0.992, 0.992, 1.0, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.984, 0.984, 0.984, 0.984, 0.984, 0.984, 0.984, 0.984, 0.984, 0.984, 0.984, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.984, 0.984, 0.984, 0.984]
[0.49245854429314884, 0.5019703776663528, 0.5082260916159203, 0.5130975501757558, 0.5013040682755924, 0.509983117336016, 0.5030687651372027, 0.4910859682113131, 0.49095953646710994, 0.4961424736980405, 0.4918502761825256, 0.506072271149805, 0.5063802974075646, 0.5085305863114334, 0.5045140704032558, 0.5009193860907593, 0.5143717027182738, 0.4947851760065075, 0.5022471043792723, 0.5021048438928243, 0.5070588873946523, 0.513012627219841, 0.5099358757

100%|██████████| 52/52 [26:19<00:00, 30.38s/it]

[0.984, 0.992, 0.992, 0.984, 0.984, 0.992, 0.992, 0.984, 0.984, 0.984, 1.0, 0.992, 0.992, 0.992, 0.992, 0.992, 1.0, 0.984, 0.992, 0.992, 0.992, 0.992, 0.992, 1.0, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.984, 0.984, 0.984, 0.984, 0.984, 0.984, 0.984, 0.984, 0.984, 0.984, 0.984, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.984, 0.984, 0.984, 0.984, 0.984]
[0.49245854429314884, 0.5019703776663528, 0.5082260916159203, 0.5130975501757558, 0.5013040682755924, 0.509983117336016, 0.5030687651372027, 0.4910859682113131, 0.49095953646710994, 0.4961424736980405, 0.4918502761825256, 0.506072271149805, 0.5063802974075646, 0.5085305863114334, 0.5045140704032558, 0.5009193860907593, 0.5143717027182738, 0.4947851760065075, 0.5022471043792723, 0.5021048438928243, 0.5070588873946523, 0.513012627219841, 0.509

In [ ]:
# save policy, q-value and state
torch.save(policy_network.state_dict(), params['log_base_dir'] + 'policy_net_final.pth')
torch.save(value_network.state_dict(), params['log_base_dir'] + 'value_net_final.pth')
torch.save(state_representation.state_dict(), params['log_base_dir'] + 'state_repr_final.pth')

In [ ]:
# save memory
with open(params['log_base_dir'] + 'memory.pickle', 'wb') as f:
  pickle.dump(train_env.memory, f)

#### Evaluation on test set

In [ ]:
with open(params['log_base_dir'] + 'memory.pickle', 'rb') as f:
  memory = pickle.load(f)

In [ ]:
# evaluate best policy on test set
state_rep = State_Representation(num_of_users, num_of_books, params['embedding_dim'], params['hidden_dim'])
policy = Actor(params['embedding_dim'], params['hidden_dim'])

state_rep.load_state_dict(torch.load(params['log_base_dir'] + 'best_state_repr.pth'))
policy.load_state_dict(torch.load(params['log_base_dir'] + 'best_policy_net.pth'))

<All keys matched successfully>

In [ ]:
hit, dcg = evaluation(policy, state_rep, memory, full_loader)
print('hit rate: ', hit, 'dcg: ', dcg)

hit rate:  0.9616925246826517 dcg:  0.47774772225701184


#### Inference

In [ ]:
random_user = test_data['user'].sample().iloc[0]
random_user

138

In [ ]:
temp_test_data = test_data[test_data.user == random_user]
temp = pd.merge(temp_test_data, filtered_df, on=['user', 'book'])
temp

,user,book,isbn,title,author,user_id,rating,best_seller
0,138,3823,679454497,Blood and Gold (Vampire Chronicles),Anne Rice,83671,8.0,True
1,138,6848,441005241,"Burnt Offerings (Anita Blake, Vampire Hunter)",Laurell K. Hamilton,83671,9.0,False
2,138,12272,316174939,The Art of Tasha Tudor,Harry Davis,83671,10.0,False
3,138,24415,60916060,The Reshaping of Everyday Life : 1790-1840 (Ev...,Jack Larkin,83671,9.0,False
4,138,26982,679759611,The Unredeemed Captive: A Family Story from Ea...,John Demos,83671,9.0,False
5,138,26986,395735270,Tasha Tudor's Heirloom Crafts,Tovah Martin,83671,10.0,True
6,138,26988,803297033,"Waheenee, an Indian Girl's Story",Gilbert L. Wilson,83671,9.0,False
7,138,27019,899661084,Smoky House,Elizabeth Goudge,83671,9.0,False
8,138,27024,441642748,The Other Glass Teat,Harlan Ellison,83671,9.0,False
9,138,27034,451096835,Approaching Oblivion,Harlan Ellison,83671,9.0,False


In [ ]:
recommendations = []

for model, state_representation in zip([policy], [state_rep]):
    env = Environment(test_matrix)
    user, memory = env.reset(random_user)

    # give 2 books that the user has rated in the past
    user, memory, reward, _ = env.step(torch.tensor([12272]))
    user, memory, reward, _ = env.step(torch.tensor([26986]))

    # get next 3 recommendations based on the two books above
    recs = []
    for i in range(3):
        action_emb = model(state_representation(user, memory))
        action = model.get_action(
            user,
            torch.tensor(env.memory[user.detach().cpu().numpy().astype(int), :]),
            state_representation,
            action_emb,
            torch.tensor([item for item in env.available_books if item not in env.viewed_books]).long()
        )
        user, memory, reward, _ = env.step(action)
        recs.append(action)

    recommendations.append(recs)

In [ ]:
recos = [r.item() for r in recommendations[0]]
recos

[8552, 3823, 17454]

In [ ]:
recommended_books = filtered_df[filtered_df.book.isin(recos)][['title', 'author', 'best_seller']].drop_duplicates()
recommended_books

,title,author,best_seller
110919,Blood and Gold (Vampire Chronicles),Anne Rice,True
166748,Beowulf : A New Verse Translation,Seamus Heaney,False
253456,His Touch,Mary Lynn Baxter,True
